常见三种任务：
* 回归问题：是预测一个连续的数值，例如明天降雨量，股票涨跌幅度等等；
* 分类问题：预测猫、狗之类的，类别Label之间没有什么大小关系；
* 排序问题：则介于回归问题和分类问题之间。Label是离散值，但不同档位的值有大小关系。
    * 例子1：用户在搜索引擎里输入一个关键词，返回一堆结果，通常会按照满足程度，对这些结果分成0,1,2,3,4五档。 同分类问题做比较，把0档分成1档 和 把0档分成2档，这种错误程度明显是不同的。这些档位在一定的大小关系，排序的目标是尽可能把3档的满足的特别好的结果排在前面。
    * 例子2：预测明天股票情况，只要能预测到明天涨幅可能最大的10只就可以，而不用关系其他几千个股票的情况。

解决排序问题的三种思路：
1. point-wise, 当成一个回归问题，预测每个结果的分值，预测得分和档位label均方误差作为损失函数，最终根据预测的分值排序；
2. pair-wise, 同一组内的样本，两两构成一个pair(a,b)，如果a>b为正样本，如果a<b为负样本, 转换为一个二分类问题，a=b?
3. list-wise，根据实际的排序情况和理想排序情况的diff，调整；

* 数据集的构造方式
* 微软rank数据集 

## 排序问题的衡量指标：
* 累计增益(CG) cumulative gain :
* 折损累计增益(DCG) Discounted CG:
* 归一化折损累计增益(NDCG) Normalized DCG:

In [20]:
# 排序问题的衡量指标:cg,dcg,ndcg
import math
import numpy as np
from sklearn.metrics import ndcg_score

#假设有5个结果的分数分别是 3、2、3、0、1、2
x = np.array([3,2,3,0,1,2])

cg = x.sum()

def dcg(scores):
    """
    compute the DCG value based on the given score
    :param scores: a score list of documents
    :return v: DCG value
    """
    v = 0
    for i in range(len(scores)):
        # val_i = scores[i]/math.log2(scores[i]+2)
        # 增加相关度影响比重的DCG计算方式
        v += (np.power(2, scores[i]) - 1) / np.log2(i+2)  # i+2 is because i starts from 0
    return v

def idcg(scores):
    """
    compute the IDCG value (best dcg value) based on the given score
    :param scores: a score list of documents
    :return:  IDCG value
    """
    best_scores = sorted(scores)[::-1]
    return dcg(best_scores)

def ndcg(scores):
    """
    compute the NDCG value based on the given score
    :param scores: a score list of documents
    :return:  NDCG value
    """
    return dcg(scores)/idcg(scores)

def ndcg_k(scores, k):
    scores_k = scores[:k]
    dcg_k = dcg(scores_k)
    idcg_k = dcg(sorted(scores)[::-1][:k])
    if idcg_k == 0:
        return np.nan
    return dcg_k/idcg_k

dcg_val = dcg(x)
idcg_val = idcg(x)
ndcg_val = ndcg(x)
print(f"dcg:{dcg_val}")
print(f"idcg:{idcg_val}")
print(f"ndcg:{ndcg_val}")

# 使用sklearn.metrics.ndcg_score
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ndcg_score.html
true_relevance = np.asarray([[10, 0, 0, 1, 5]]) #真实的
scores = np.asarray([[.1, .2, .3, 4, 70]]) #预测的

ndcg = ndcg_score(true_relevance, scores)
print(f"ndcg:{ndcg}")
ndcg2 = ndcg_score(np.asarray([[3,3,2,2,1,0]]), np.asarray([[3,3,2,2,1,0]]))
print(f"ndcg2:{ndcg2}\n")

dcg:13.848263629272981
idcg:14.595390756454924
ndcg:0.9488107485678985
ndcg:0.6956940443813076
ndcg2:0.9999999999999998



# 1. point-wise

模型跟普通的回归问题一样

# 2. Pair-wise
https://www.cnblogs.com/little-horse/p/10468311.html

同一组内，两两构造一个pair对，训练模型时，输入是pair对的两个向量，输出是三种a>b label为1,a=b label为0 (这个需要吗？)，a<b, label为-1。 预测时，只使用ab比较前的网络。

缺点：
1. 样本数量将会增加很多倍 
2. 对噪声标注更敏感，即一个错误标注会引起多个 pair 标注错误 
3. 仅考虑了 pair 的相对位置，同样也没有考虑样本在预测排序中的位置信息

In [3]:
import torch
import torch.nn as nn

class RankNet(nn.Module):
    def __init__(self, num_features,hidden=[512,256,128]):
        # hidden 的数值为超参数，可通过Gridsearch确定一个合适值
        super(RankNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features,hidden[0]), 
            nn.Dropout(0.2), 
            nn.ReLU(),
            nn.Linear(hidden[0], hidden[1]), 
            nn.Dropout(0.2),
            nn.ReLU(), 
            nn.Linear(hidden[1], hidden[2]),
            nn.Dropout(0.2), 
            nn.ReLU(),
            nn.Linear(hidden[2], 1))
        self.sigmoid = nn.Sigmoid()

    def forward(self, input1, input2):
        # 训练过程中，接受pair对中的两个向量，通过model获取得分，两个得分差值过sigmoid函数，用于bce计算损失
        s1 = self.model(input1)
        s2 = self.model(input2)
        diff = s1 - s2
        prob = self.sigmoid(diff)
        return prob

    def predict(self, input_):
        # 实际预测的时候，只需要输入一个，得到得分即可
        return self.model(input_)

model = RankNet(10)
criterion = nn.BCELoss() #使用bce损失函数

# 2. List-wise
LambdaRank, NDCG非凸且不可导, 损失函数或者梯度该怎么定义？

* https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf
* https://papers.nips.cc/paper/2971-learning-to-rank-with-nonsmooth-cost-functions.pdf
* https://github.com/haowei01/pytorch-examples/blob/master/ranking/LambdaRank.py
* https://github.com/airalcorn2/RankNet/blob/master/lambdarank.py

* https://github.com/houchenyu/L2R/blob/master/lambdaRank.py

In [4]:
class LambdaRankModule(nn.Module):
    def __init__(self, num_features,hidden=[128,64,32]):
        super(LambdaRankModule, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features,hidden[0]), 
            nn.Dropout(0.2), 
            nn.ReLU(),
            nn.Linear(hidden[0], hidden[1]), 
            nn.Dropout(0.2),
            nn.ReLU(), 
            nn.Linear(hidden[1], hidden[2]),
            nn.Dropout(0.2), 
            nn.ReLU(),
            nn.Linear(hidden[2], 1))
        
        
    def forward(self, input1):
        s1 = self.model(input1)
        return s1 

n_docs = 8
n_rel = 5 #
n_irr = n_docs - n_rel

model = LambdaRankModule(10)
docs = torch.rand(n_docs,10) #n_docs-batch,10-num_features
# print(docs)
doc_scores = model(docs)
# print(doc_scores)

* LambdaRank是一个经验算法，不通过显示定义损失函数再求梯度的方式对排序问题进行求解，而是分析排序问题需要的梯度的意义，直接定义梯度。
* Lambda量化了一个待排序的文档在下一次迭代时应该调整的方向和强度。
* 通过交换每对文档后 NDCG 的变化来衡量梯度.
* |ΔNDCG|是交换排序结果Ui和Uj得到的NDCG差值。NDCG倾向于将排名高并且相关性高的文档更快地向上推动，而排名地而且相关性较低的文档较慢地向上推动。
* https://www.cnblogs.com/genyuan/p/9788294.html
* 直接把 RankNet 最后得到的 Lambda 梯度拿来作为 LambdaRank 的梯度来用了？
![test2](https://www.zhihu.com/equation?tex=++++%5Clambda_%7Bij%7D%5Cmathrel%7B%5Cstackrel%7B%5Cmbox%7Bdef%7D%7D%7B%3D%7D%7D+-+%5Cfrac%7B1%7D%7B1+%2B+%5Cexp%5Cbigl%28s_i+-+s_j%5Cbigr%29%7D+%5C%5C)
![test](https://www.zhihu.com/equation?tex=++++%5Clambda_%7Bij%7D+%3D+-+%5Cfrac%7B1%7D%7B1+%2B+%5Cexp%5Cbigl%28s_i+-+s_j%5Cbigr%29%7D+%5Ccdot+%5Clvert%5CDelta+Z_%7Bij%7D%5Crvert+%5C%5C)

In [5]:
# def idcg(n_rel):
#     # Assuming binary relevance. 这个假设有点奇怪
#     nums = np.ones(n_rel)
#     denoms = np.log2(np.arange(n_rel) + 1 + 1)
#     return (nums / denoms).sum()

def c_lambda():
    # Document ranks.
    (sorted_scores, sorted_idxs) = doc_scores.sort(dim=0, descending=True) #对预测得分从大到小排序
    doc_ranks = torch.zeros(n_docs)
    doc_ranks[sorted_idxs] = 1 + torch.arange(n_docs).view((n_docs, 1)).float()
    doc_ranks = doc_ranks.view((n_docs, 1))

    # Compute lambdas.
    # See equation (6) in [2] and equation (9) in [1].
    score_diffs = doc_scores[:n_rel] - doc_scores[n_rel:].view(n_irr)
    # print(doc_scores[:n_rel])
    # print(doc_scores[n_rel:].view(n_irr))
    exped = score_diffs.exp()
    N = 1 / idcg(n_rel)
    dcg_diffs = 1 / (1 + doc_ranks[:n_rel]).log2() - (1 / (1 + doc_ranks[n_rel:]).log2()).view(n_irr)
    lamb_updates = 1 / (1 + exped) * N * dcg_diffs.abs()
    # print(lamb_updates)
    lambs = torch.zeros((n_docs, 1))
    lambs[:n_rel] += lamb_updates.sum(dim=1, keepdim=True)
    lambs[n_rel:] -= lamb_updates.sum(dim=0, keepdim=True).t()

In [22]:
#https://github.com/houchenyu/L2R/blob/master/lambdaRank.py  
def single_dcg(scores, i, j):
    """
    compute the single dcg that i-th element located j-th position
    :param scores:
    :param i:
    :param j:
    :return:
    """
    return (np.power(2, scores[i]) - 1) / np.log2(j+2)

def compute_lambda(true_scores, predict_scores, order_pairs, qid):
    """
    :param true_scores: the score list of the documents for the qid query
    :param predict_scores: the predict score list of the these documents
    :param order_pairs: the partial oder pairs where first document has higher score than the second one
    :param qid: specific query id
    :return:
        lambdas: changed lambda value for these documents
        w: w value
        qid: query id
    """
    doc_num = len(true_scores)
    lambdas = np.zeros(doc_num)
    w = np.zeros(doc_num)
    IDCG = idcg(true_scores)
    single_dcgs ={}
    for i, j in order_pairs:
        if (i, i) not in single_dcgs:
            single_dcgs[(i, i)] = single_dcg(true_scores, i, i)
        if (j, j) not in single_dcgs:
            single_dcgs[(j, j)] = single_dcg(true_scores, j, j)
        single_dcgs[(i, j)] = single_dcg(true_scores, i, j)
        single_dcgs[(j, i)] = single_dcg(true_scores, j, i)

    for i, j in order_pairs:
        delta = abs(single_dcgs[(i,j)] + single_dcgs[(j,i)] - single_dcgs[(i,i)] -single_dcgs[(j,j)])/IDCG
        rho = 1 / (1 + np.exp(predict_scores[i] - predict_scores[j]))
        lambdas[i] += rho * delta
        lambdas[j] -= rho * delta

        rho_complement = 1.0 - rho
        w[i] += rho * rho_complement * delta
        w[j] -= rho * rho_complement * delta

    return lambdas, w, qid